# Aprendizaje automático (Machine Learning)

### Inteligencia Artificial
### Universidad de Sevilla

#### Ejercicio 1. RECOPILACIÓN DE DATOS

Para leer los datos desde el fichero `cars.csv` que se proporciona se pueden evaluar las siguientes expresiones ([_Pandas_](http://pandas.pydata.org/) y [_NumPy_](http://www.numpy.org/) son paquetes de _Python_ para análisis de datos y cálculo científico, respectivamente):

In [29]:
import pandas 
#import numpy
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob


In [5]:
tweet_fifa_prev = pandas.read_csv('fifa_tweets_emotion.csv', header=None,
                       names=['number', 'date', 'number_likes','source', 'tweet','feeling'])

## para sacar estos datos de tablas --> Kaggle (hay que hacer su cuenta) --> Search: por ejemplo, spam detection o
## feelins detection (10000 datos)

In [6]:
# Número de filas y columnas
print(tweet_fifa_prev.shape)

(22525, 6)


In [7]:
# Muestra las 10 primeras filas
tweet_fifa_prev.head(5)

,number,date,number_likes,source,tweet,feeling
0,NaN,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
1,0.0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
2,1.0,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
3,2.0,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
4,3.0,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive


In [8]:
delete_col=["number", "date", "number_likes", "source", "feeling"]
tweet_fifa_col= tweet_fifa_prev.drop(delete_col, axis=1)
tweet_fifa=tweet_fifa_col.drop(0, axis=0)
tweet_fifa.head(5)


,tweet
1,What are we drinking today @TucanTribe \n@MadB...
2,Amazing @CanadaSoccerEN #WorldCup2022 launch ...
3,Worth reading while watching #WorldCup2022 htt...
4,Golden Maknae shinning bright\n\nhttps://t.co/...
5,"If the BBC cares so much about human rights, h..."


#### Ejercicio 2. LIMPIEZA DEL TEXTO, ELIMINAR LAS PALABRAS QUE NO APORTAN INFORMACIÓN.

In [9]:
def limpiar_texto(data_set):
    patron_emoticonos = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # Emoticonos generales
                                   u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                                   u"\U0001F680-\U0001F6FF"  # Transporte y mapas
                                   u"\U0001F700-\U0001F77F"  # Símbolos de alquimia
                                   u"\U0001F780-\U0001F7FF"  # Formas geométricas extendidas
                                   u"\U0001F800-\U0001F8FF"  # Símbolos de suplemento adicional
                                   u"\U0001F900-\U0001F9FF"  # Emoticonos de personas y cuerpos
                                   u"\U0001FA00-\U0001FA6F"  # Símbolos de objetos
                                   u"\U0001FA70-\U0001FAFF"  # Símbolos de alimentos
                                   u"\U00002702-\U000027B0"  # Otros emoticonos y símbolos
                                   "]+", flags=re.UNICODE)
    tokenizer=TweetTokenizer()
    nltk.download("stopwords")
    stopwords_english = stopwords.words("english")
    stemmer = SnowballStemmer("english")
    diccionario = {"tweet": []}
    for index, fila in data_set.iterrows():
        
        tweet=fila["tweet"]
        # Eliminar el símbolo #
        tweet = re.sub(r'#', '', tweet)
    
        # Eliminar el símbolo @
        tweet = re.sub(r'@\S+', '', tweet)
    
        # Eliminar URLs (patrón básico para ilustrar)
        tweet = re.sub(r'http\S+', '', tweet) ##\S+ que elimine tambien lo que esté pegado
        
        # Eliminar emoticonos
        #tweet = patron_emoticonos.sub(r'',tweet)
        
        # Poner en minuscula
        tweet=tweet.lower()
        
        tweet= tokenizer.tokenize(tweet)
        list_tweet=[]
        for palabra in tweet:
            if palabra not in stopwords_english:
                palabra_proc = stemmer.stem(palabra)
                list_tweet.append(palabra_proc)
        
        cadena_tweet=""
        for palabra in list_tweet:
            cadena_tweet+=" "
            cadena_tweet+=palabra
            
        diccionario["tweet"].append(cadena_tweet)
    data_set_proc=pandas.DataFrame(diccionario)
    return data_set_proc

In [10]:
tweet_fifa_limp=limpiar_texto(tweet_fifa)
tweet_fifa_limp.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet
0,drink today worldcup 2022
1,amaz worldcup 2022 launch video . show much f...
2,worth read watch worldcup 2022
3,golden makna shin bright jeonjungkook jungkoo...
4,"bbc care much human right , homosexu right , ..."


#### Ejercicio 3. ETIQUETADO DE DATOS CON HERRAMIENTAS YA EXISTENTES

In [11]:
def clasificador(data_set):
    diccionario={"tweet":[], "sentimiento":[]}
    for index, fila in data_set.iterrows():
        tweet=fila["tweet"]
        texto=TextBlob(tweet)
        sentimiento_pol=texto.sentiment.polarity
        if sentimiento_pol < (-0.6):
            sentimiento="Hater"
        elif (-0.6) <= sentimiento_pol < (-0.2):
            sentimiento="Molesto"
        elif (-0.2) <= sentimiento_pol < 0.2:
            sentimiento="Neutro"
        elif 0.2 <= sentimiento_pol < 0.6:
            sentimiento="Contento"
        else:
            sentimiento="Muy feliz"
        diccionario["tweet"].append(tweet)
        diccionario["sentimiento"].append(sentimiento)
    data_set_sent=pandas.DataFrame(diccionario)
    return data_set_sent

In [12]:
tweet_fifa_sent=clasificador(tweet_fifa_limp)
tweet_fifa_sent.head(5)

,tweet,sentimiento
0,drink today worldcup 2022,Neutro
1,amaz worldcup 2022 launch video . show much f...,Neutro
2,worth read watch worldcup 2022,Contento
3,golden makna shin bright jeonjungkook jungkoo...,Contento
4,"bbc care much human right , homosexu right , ...",Neutro


#### Ejercicio 4. CODIFICACIÓN DE LOS ATRIBUTOS Y OBJETIVOS.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

atributos=tweet_fifa_sent["tweet"]


## vectorizer=CountVectorizer() --> solo cuenta las veces que aparece una palabra en el texto
## atributos=vectorizer.fit_transform(atributos)

# Entrenamiento de Word2Vec
atributos = Word2Vec([atributos], vector_size=100, window=5, min_count=1, worker=4)

# Obtención del embedding de una palabra
embedding = model.wv['amaz']
print(embedding)
#objetivo=tweet_fifa_sent["sentimiento"]

#(atributos_entrenamiento, atributos_prueba,
# objetivo_entrenamiento, objetivo_prueba) = train_test_split(
 #       atributos, objetivo,
  #      random_state=12345,
  #      test_size=.2,
   #     stratify=objetivo)
#print(atributos_entrenamiento)
#print(atributos_prueba)

In [5]:
atributos = cars.loc[:, 'buying':'safety']  # selección de las columnas de atributos, con el .loc es pq le estoy dando unas 
                                            # coordenadas específicas.
objetivo = cars['acceptability']  # selección de la columna objetivo, debe ser el resultado de la clasificacion

In [7]:
# Para realizar una codificación de los datos, se crea una instancia del tipo de
# codificación pretendida y se ajusta a los datos disponibles mediante el método fit.

# El codificador adecuado para los atributos es OrdinalEncoder, ya que permite
# trabajar con el array completo de valores de los atributos.

#Crear instancia:
codificador_atributos = preprocessing.OrdinalEncoder()

# lo entreno... (para que sepa cuantas categorías hay, en estos datos concretos)
codificador_atributos.fit(atributos)

# Categorías detectadas por el codificador para cada atributo
print(codificador_atributos.categories_)

[array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['2', '3', '4', '5more'], dtype=object), array(['2', '4', 'more'], dtype=object), array(['big', 'med', 'small'], dtype=object), array(['high', 'low', 'med'], dtype=object)]


In [8]:
# Una vez ajustado el codificador, el método transform permite codificar los
# valores de los atributos
atributos_codificados = codificador_atributos.transform(atributos)
print(atributos_codificados)

[[3. 3. 0. 0. 2. 1.]
 [3. 3. 0. 0. 2. 2.]
 [3. 3. 0. 0. 2. 0.]
 ...
 [1. 1. 3. 2. 0. 1.]
 [1. 1. 3. 2. 0. 2.]
 [1. 1. 3. 2. 0. 0.]]


In [11]:
# El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
# con una lista o array unidimensional de sus valores
codificador_objetivo = preprocessing.LabelEncoder()

# El método fit_transform ajusta la codificación y la aplica a los datos justo
# a continuación
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)

Una vez codificadas las variables, es necesario separar el conjunto de datos en dos: un conjunto de entrenamiento, que se usará para construir los distintos modelos; y un conjunto de prueba, que se usará para comparar los distintos modelos.

Un detalle a tener en cuenta es que la distribución de ejemplos en las distintas clases de aceptabilidad no es uniforme: hay 1210 coches (un 70.023&nbsp;% del total) clasificados como inaceptables (`unacc`), 384 coches (22.222&nbsp;%) clasificados como aceptables (`acc`), 69 coches (3.993&nbsp;%) clasificados como buenos (`good`) y 65 coches (3.762&nbsp;%) clasificados como muy buenos (`vgood`).

Tengo que mantener las proporciones

In [15]:
# Frecuencia total de cada clase de aceptabilidad, objtetivo sigue siendo una tabla pero de una sola columna.
print(pandas.Series(objetivo).value_counts(normalize=True))

unacc    0.700231
acc      0.222222
good     0.039931
vgood    0.037616
Name: acceptability, dtype: float64


Es conveniente, por tanto, que la separación de los ejemplos se realice de manera estratificada, es decir, intentando mantener la proporción anterior tanto en el conjunto de entrenamiento como en el de prueba.

Para dividir un conjunto de datos en un subconjunto de entrenamiento y otro de prueba, _sklearn_ proporciona la función `train_test_split`.

In [9]:
from sklearn import model_selection

In [12]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_codificados, objetivo_codificado,
        # Valor de la semilla aleatoria, para que el muestreo sea reproducible,
        # a pesar de ser aleatorio --> mas numero, más barajas la baraja, más random es.
        random_state=12345,
        # Tamaño del conjunto de prueba
        test_size=.33,
        # Estratificamos respecto a la distribución de valores en la variable objetivo
        stratify=objetivo_codificado)

In [13]:
# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print('Cantidad de ejemplos de pruebas requeridos:', 1728 * .33)
print('Filas del array de atributos de prueba:', atributos_prueba.shape[0])
print('Longitud del vector de objetivos de prueba:', len(objetivo_prueba))
print('Proporción de clases en el vector de objetivos de prueba:')
print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_prueba)
      ).value_counts(normalize=True))

Cantidad de ejemplos de pruebas requeridos: 570.24
Filas del array de atributos de prueba: 571
Longitud del vector de objetivos de prueba: 571
Proporción de clases en el vector de objetivos de prueba:
unacc    0.700525
acc      0.222417
good     0.040280
vgood    0.036778
Name: proportion, dtype: float64


In [19]:
# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print('Cantidad de ejemplos de entrenamiento requeridos:', 1728 * .67)
print('Filas del array de atributos de entrenamiento:', atributos_entrenamiento.shape[0])
print('Longitud del vector de objetivos de entrenamiento:', len(objetivo_entrenamiento))
print('Proporción de clases en el vector de objetivos de entrenamiento:')
print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_entrenamiento)
      ).value_counts(normalize=True))

Cantidad de ejemplos de entrenamiento requeridos: 1157.76
Filas del array de atributos de entrenamiento: 1157
Longitud del vector de objetivos de entrenamiento: 1157
Proporción de clases en el vector de objetivos de entrenamiento:
unacc    0.700086
acc      0.222126
good     0.039758
vgood    0.038029
dtype: float64


Para realizar aprendizaje supervisado en _sklearn_, basta crear una instancia de la clase de objetos que implemente el modelo que se quiera utilizar (_naive_ Bayes, árboles de decisión, _kNN_, etc.).

Cada una de estas instancias dispondrá de los siguientes métodos:
* El método `fit` permite entrenar el modelo, dados __por separado__ el conjunto de ejemplos de entrenamiento y la clase de cada uno de estos ejemplos.
* El método `predict` permite clasificar un nuevo ejemplo una vez entrenado el modelo.
* El método `score` calcula el rendimiento del modelo, dados __por separado__ el conjunto de ejemplos de prueba y la clase de cada uno de estos ejemplos.

### _Naive_ Bayes

_sklearn_ implementa _naive_ Bayes para atributos categóricos mediante instancias de la clase `CategoricalNB`. Para otro tipo de tareas, como la que se presentá más adelante con procesamiento del lenguaje natural, se usa MultinomialNB.

In [14]:
from sklearn import naive_bayes

In [15]:
clasif_NB = naive_bayes.CategoricalNB(alpha=1.0)
clasif_NB.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB()

El método `score` la tasa de acierto (_accuracy_) sobre un conjunto de datos de prueba. 

In [26]:
clasif_NB.score(atributos_prueba, objetivo_prueba)
##pal examen por lo menos un 0.70; o mejorar el modelo (version que te viene mejor) o tu conjunto de datos.

0.8441330998248686

El método `predict` devuelve la clase predicha por el modelo para usarlo en un futuro con un nuevo ejemplo.